# Comvolution / Pooling レイヤの実装

## 4次元配列

In [1]:
import sys, os
sys.path.append(os.path.abspath(os.path.join('..', 'sample')))
import numpy as np

In [2]:
x = np.random.rand(10, 1, 28, 28) # ランダムにデータを生成
x.shape

(10, 1, 28, 28)

In [3]:
x[0].shape # 1つ目のデータにアクセス

(1, 28, 28)

In [4]:
x[1].shape # 2つ目のデータにアクセス

(1, 28, 28)

In [5]:
x[0, 0] # 1つ目のデータの、1チャンネル目の空間データにアクセス

array([[7.46455120e-01, 2.27963876e-01, 8.27426402e-01, 8.26803413e-01,
        8.60531394e-01, 5.11418231e-01, 5.27047620e-01, 7.03399015e-01,
        7.32704811e-02, 1.09308993e-01, 8.15014474e-01, 3.14114566e-01,
        9.24025136e-01, 2.80541838e-01, 6.79584986e-03, 1.10792427e-01,
        4.48246393e-01, 8.82760812e-02, 9.05718234e-01, 7.86133933e-01,
        2.55209058e-01, 5.22796916e-01, 7.02683868e-01, 1.94566704e-01,
        6.36620563e-01, 5.16364544e-01, 3.03474860e-01, 2.11320952e-01],
       [4.03951927e-01, 1.87770854e-01, 5.80080664e-01, 8.34256457e-01,
        9.39314094e-01, 2.45657250e-01, 8.65530830e-01, 9.20854345e-01,
        1.56331998e-01, 7.62897476e-01, 8.67818851e-01, 3.70141368e-01,
        2.84505630e-01, 3.78607312e-01, 5.58315978e-01, 9.20036356e-01,
        2.58880224e-01, 9.56287150e-01, 2.20749950e-01, 6.86061026e-01,
        1.04820287e-01, 4.47132139e-01, 7.62985191e-01, 8.12218572e-01,
        3.59674603e-01, 2.05085311e-01, 8.31086229e-01, 9.64650

## im2colによる展開
- NumPyでは、`for`分を使うと処理が遅くなってしまう
- `for`分を使わず、`im2col`を使う

`im2col`は image to columnの略、「画像から行列へ」

## Convolution レイヤの実装

In [6]:
import sys, os
sys.path.append(os.path.abspath(os.path.join('..', 'sample')))
import numpy as np
from common.util import im2col

# パッチサイズが1, チャンネル3の 7x7のデータ
x1 = np.random.rand(1, 3, 7, 7)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(f'col1.shape: {col1.shape}')

# パッチサイズが10, チャンネル3の 7x7のデータ
x2 = np.random.rand(10, 3, 7, 7)
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(f'col2.shape: {col2.shape}')

col1.shape: (9, 75)
col2.shape: (90, 75)


In [7]:
class Comvolution:
    """
    畳み込み層
    """
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
        
        # 中間データ（backward時に使用）
        self.x = None   
        self.col = None
        self.col_W = None
        
        # 重み・バイアスパラメータの勾配
        self.dW = None
        self.db = None

    def forward(self, x):
        """
        順伝搬
        """
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1 + (H + 2*self.pad - FH) / self.stride) # 出力データの縦幅
        out_w = int(1 + (W + 2*self.pad - FW) / self.stride) # 出力データの横幅
        
        col = im2col(x, FH, FW, self.stride, self.pad) # 入力データの展開
        col_W = self.W.reshape(FN, -1).T               # フィルターを2次元に展開
        out = np.dot(col, col_W) + self.b              # 展開した行列の籍を計算
        
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)
        
        self.x = x
        self.col = col
        self.col_W = col_W

        return out

    def backward(self, dout):
        """
        逆伝搬
        """
        FN, C, FH, FW = self.W.shape
        dout = dout.transpose(0,2,3,1).reshape(-1, FN)

        self.db = np.sum(dout, axis=0)
        self.dW = np.dot(self.col.T, dout)
        self.dW = self.dW.transpose(1, 0).reshape(FN, C, FH, FW)

        dcol = np.dot(dout, self.col_W.T)
        dx = col2im(dcol, self.x.shape, FH, FW, self.stride, self.pad)

        return dx

## Poolingレイヤの実装

In [8]:
class Pooling:
    """
    プーリング層
    """
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
        self.x = None
        self.arg_max = None

    def forward(self, x):
        """
        順伝搬
        """
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride) # 出力データの縦幅
        out_w = int(1 + (W - self.pool_w) / self.stride) # 出力データの横幅

        # 入力データの展開
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h*self.pool_w)

        # 行ごとに最大値をとる
        arg_max = np.argmax(col, axis=1)
        out = np.max(col, axis=1)
        
        # 適切な出力サイズに整形する
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)

        self.x = x
        self.arg_max = arg_max

        return out

    def backward(self, dout):
        """
        逆伝搬
        """
        dout = dout.transpose(0, 2, 3, 1)
        
        pool_size = self.pool_h * self.pool_w
        dmax = np.zeros((dout.size, pool_size))
        dmax[np.arange(self.arg_max.size), self.arg_max.flatten()] = dout.flatten()
        dmax = dmax.reshape(dout.shape + (pool_size,)) 
        
        dcol = dmax.reshape(dmax.shape[0] * dmax.shape[1] * dmax.shape[2], -1)
        dx = col2im(dcol, self.x.shape, self.pool_h, self.pool_w, self.stride, self.pad)
        
        return dx

In [24]:
import sys, os
sys.path.append(os.path.abspath(os.path.join('..', 'sample')))
import numpy as np
from common.util import im2col
from common.util import col2im

class Pooling:
    """
    プーリング層
    """
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
        self.x = None
        self.arg_max = None

    def forward(self, x):
        """
        順伝搬
        """
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride) # 出力データの縦幅
        out_w = int(1 + (W - self.pool_w) / self.stride) # 出力データの横幅

        # 入力データの展開
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h*self.pool_w)

        # 行ごとに最大値をとる
        arg_max = np.argmax(col, axis=1)
        out = np.max(col, axis=1)
        
        # 適切な出力サイズに整形する
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)

        self.x = x
        self.arg_max = arg_max

        return out

    def backward(self, dout):
        """
        逆伝搬
        """
        dout = dout.transpose(0, 2, 3, 1)
        
        pool_size = self.pool_h * self.pool_w
        dmax = np.zeros((dout.size, pool_size))
        dmax[np.arange(self.arg_max.size), self.arg_max.flatten()] = dout.flatten()
        
        print("----reshape.1----")
        print(f"dmax.shape: {dmax.shape}")
        print(f"dout.shape: {dout.shape}")
        print(f"(pool_size,): {(pool_size,)}")
        dmax = dmax.reshape(dout.shape + (pool_size,)) 
        print(f"dmax.shape: {dmax.shape}")
        
        print("----reshape.2----")
        print(f"dmax.shape[0]: {dmax.shape[0]}")
        print(f"dmax.shape[1]: {dmax.shape[1]}")
        print(f"dmax.shape[2]: {dmax.shape[2]}")
        dcol = dmax.reshape(dmax.shape[0] * dmax.shape[1] * dmax.shape[2], -1)
        print(f"dcol.shape: {dcol.shape}")

        dx = col2im(dcol, self.x.shape, self.pool_h, self.pool_w, self.stride, self.pad)
        
        return dx
    
pool = Pooling(2, 2, stride=2)
x = np.random.rand(1, 2, 16, 32)
print(f"x.shape: {x.shape}")

print()
dout = pool.forward(x)
print(f"dout.shape: {dout.shape}")

print()
inX = pool.backward(dout)
print(f"inX.shape: {inX.shape}")

x.shape: (1, 2, 16, 32)

dout.shape: (1, 2, 8, 16)

----reshape.1----
dmax.shape: (256, 4)
dout.shape: (1, 8, 16, 2)
(pool_size,): (4,)
dmax.shape: (1, 8, 16, 2, 4)
----reshape.2----
dmax.shape[0]: 1
dmax.shape[1]: 8
dmax.shape[2]: 16
dcol.shape: (128, 8)
inX.shape: (1, 2, 16, 32)
